# EvoGFuzz++

In [1]:
import string
import math

from fuzzingbook.Grammars import Grammar
from isla.language import DerivationTree

GRAMMAR: Grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"]
}

INITIAL_INPUTS = ['cos(10)', 'sqrt(28367)', 'tan(-12)', 'sqrt(-900)']

Let's parse the initial input files an observe if they are valid according to the grammar

In [2]:
from fuzzingbook.Parser import EarleyParser, tree_to_string

p = EarleyParser(GRAMMAR)

for inp in INITIAL_INPUTS:
    for tree in p.parse(inp):
        assert tree_to_string(tree) == inp, f"{tree} != {inp}"

In [3]:
def arith_eval(inp: DerivationTree) -> float:
    return eval(str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan})

# Returns true if a Error is found
def prop(inp: DerivationTree) -> bool:
    try:
        arith_eval(inp)
        return False
    except ValueError:
        return True


In [4]:
print([(x, prop(x)) for x in INITIAL_INPUTS])

[('cos(10)', False), ('sqrt(28367)', False), ('tan(-12)', False), ('sqrt(-900)', True)]


In [5]:
from evogfuzzpp.evogfuzzpp import EvoGFuzz
from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s:  %(message)s"
)

In [6]:
epp = EvoGFuzz(GRAMMAR, prop, INITIAL_INPUTS, working_dir=None)

In [7]:
epp.execute()

2022-12-21 15:07:44,645:  Executing EvoGFuzz
2022-12-21 15:07:44,647:  Executing input files
2022-12-21 15:07:44,649:  Learned probabilistic grammar:
2022-12-21 15:07:44,649:  <start>                                 [('<arith_expr>', {'prob': None})]
2022-12-21 15:07:44,650:  <arith_expr>                            [('<function>(<number>)', {'prob': None})]
2022-12-21 15:07:44,650:  <function>                              [('sqrt', {'prob': 1.0}), ('sin', {'prob': 0.0}), ('cos', {'prob': 0.0}), ('tan', {'prob': 0.0})]
2022-12-21 15:07:44,651:  <number>                                [('<maybe_minus><onenine><maybe_digits><maybe_frac>', {'prob': None})]
2022-12-21 15:07:44,652:  <maybe_minus>                           [('', {'prob': 0.0}), ('-', {'prob': 1.0})]
2022-12-21 15:07:44,653:  <onenine>                               [('1', {'prob': 0.0}), ('2', {'prob': 0.0}), ('3', {'prob': 0.0}), ('4', {'prob': 0.0}), ('5', {'prob': 0.0}), ('6', {'prob': 0.0}), ('7', {'prob': 0.0}), ('8', {'

cos(10)
sqrt(28367)
tan(-12)
sqrt(-900)
sqrt(-900000)
sqrt(-9000.00)
sqrt(-900000000.0)
sqrt(-90000)
sqrt(-900000.0)
sqrt(-90000000)
sqrt(-900.000)
sqrt(-90000000000)
sqrt(-90000.0000)
sqrt(-900.0)
sqrt(-900.00000)
sqrt(-9000.0)
sqrt(-9000)
sqrt(-900.0000)
sqrt(-90)
sqrt(-900)
sqrt(-9000000)
sqrt(-900.00)
sqrt(-90.0000)
sqrt(-90.00)
sqrt(-90.000)
sqrt(-90.00000)
sqrt(-90.0)
('sqrt(-900000)', True)
('sqrt(-9000.00)', True)
('sqrt(-900000000.0)', True)
('sqrt(-90000)', True)
('sqrt(-900000.0)', True)
('sqrt(-90000000)', True)
('sqrt(-900.000)', True)
('sqrt(-90000000000)', True)
('sqrt(-90000.0000)', True)
('sqrt(-900.0)', True)
('sqrt(-900.00000)', True)
('sqrt(-9000.0)', True)
('sqrt(-9000)', True)
('sqrt(-900.0000)', True)
('sqrt(-90)', True)
('sqrt(-900)', True)
('sqrt(-9000000)', True)
('sqrt(-900.00)', True)
('sqrt(-90.0000)', True)
('sqrt(-90.00)', True)
('sqrt(-90.000)', True)
('sqrt(-90.00000)', True)
('sqrt(-90.0)', True)


NameError: name 'exec_data' is not defined